In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

import os
import sys

project_path=(os.path.join(os.getcwd(),'..','..'))
sys.path.append(project_path)

###AUTO LOADER

In [0]:
df_user = (
    spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "parquet")
        .option(
            "cloudFiles.schemaLocation",
            "abfss://silver@hariazureproject.dfs.core.windows.net/DimUser/schema"
        )
        .option(
            "checkpointLocation",
            "abfss://silver@hariazureproject.dfs.core.windows.net/DimUser/checkpoint"
        )
        .load("abfss://bronze@hariazureproject.dfs.core.windows.net/DimUser")
)


In [0]:
# display(
#     df_user,
#     checkpointLocation="abfss://silver@hariazureproject.dfs.core.windows.net/DimUser/checkpoint"
# )


In [0]:
df_user = df_user.withColumn("user_name",upper(col('user_name')))

In [0]:
# display(
#     df_user,
#     checkpointLocation="abfss://silver@hariazureproject.dfs.core.windows.net/DimUser/checkpoint"
# )


## Drop duplicates

In [0]:
df_user = df_user.dropDuplicates(['user_id'])

In [0]:
df_user.isStreaming


True

In [0]:
df_user.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option(
        "checkpointLocation",
        "abfss://silver@hariazureproject.dfs.core.windows.net/DimUser/checkpoint"
    ) \
    .trigger(once=True) \
    .option("path","abfss://silver@hariazureproject.dfs.core.windows.net/DimUser/data")\
    .toTable("spotify_cata.silver.DimUser")

    # .start(
    #     "abfss://silver@hariazureproject.dfs.core.windows.net/DimUser/data"
    # )


#DimArtist

In [0]:
df_artist = (
    spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "parquet")
        .option(
            "cloudFiles.schemaLocation",
            "abfss://silver@hariazureproject.dfs.core.windows.net/DimArtist/schema"
        )
        .option(
            "checkpointLocation",
            "abfss://silver@hariazureproject.dfs.core.windows.net/DimArtist/checkpoint"
        )
        .load("abfss://bronze@hariazureproject.dfs.core.windows.net/DimArtist")
)


In [0]:
# display(
#     df_artist,
#     checkpointLocation="abfss://silver@hariazureproject.dfs.core.windows.net/DimArtist/checkpoint"
# )


In [0]:
df_artist.isStreaming


True

In [0]:
df_artist = df_artist.dropDuplicates(['artist_id'])

In [0]:
%sql
GRANT USE CATALOG ON CATALOG spotify_cata TO `arigeharikrishna3@gmail.com`;
GRANT USE SCHEMA ON SCHEMA spotify_cata.silver TO `arigeharikrishna3@gmail.com`;
GRANT CREATE SCHEMA 
ON CATALOG spotify_cata 
TO `arigeharikrishna3@gmail.com`;
-- GRANT MODIFY ON TABLE spotify_cata.silver.DimUser TO `arigeharikrishna3@gmail.com`;


In [0]:
%sql
GRANT CREATE TABLE ON SCHEMA spotify_cata.silver TO `arigeharikrishna3@gmail.com`;


In [0]:
df_artist.writeStream.format("delta") \
    .outputMode("append") \
    .option(
        "checkpointLocation",
        "abfss://silver@hariazureproject.dfs.core.windows.net/DimArtist/checkpoint"
    ) \
    .trigger(once=True) \
    .option("path","abfss://silver@hariazureproject.dfs.core.windows.net/DimArtist/data")\
    .toTable("spotify_cata.silver.DimArtist")
    # .start(
    #     "abfss://silver@hariazureproject.dfs.core.windows.net/DimArtist/data"
    # )


#DimTrack


In [0]:
df_track = (
    spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "parquet")
        .option(
            "cloudFiles.schemaLocation",
            "abfss://silver@hariazureproject.dfs.core.windows.net/DimTrack/schema"
        )
        .option(
            "checkpointLocation",
            "abfss://silver@hariazureproject.dfs.core.windows.net/DimTrack/checkpoint"
        )
        .load("abfss://bronze@hariazureproject.dfs.core.windows.net/DimTrack")
)


In [0]:
# display(
#     df_track,
#     checkpointLocation="abfss://silver@hariazureproject.dfs.core.windows.net/DimTrack/checkpoint"
# )


In [0]:
df_track = df_track.withColumn("durationFlag",when(col('duration_sec')<150,"low")\
                                .when(col('duration_sec')<300,"medium")\
                                .otherwise("high"))

df_track = df_track.withColumn("track_name",regexp_replace(col('track_name'),'-',' '))


In [0]:
df_track.writeStream.format("delta") \
    .outputMode("append") \
    .option(
        "checkpointLocation",
        "abfss://silver@hariazureproject.dfs.core.windows.net/DimTrack/checkpoint"
    ) \
    .trigger(once=True) \
    .option("path","abfss://silver@hariazureproject.dfs.core.windows.net/DimTrack/data")\
    .toTable("spotify_cata.silver.DimTrack")
    # .start(
    #     "abfss://silver@hariazureproject.dfs.core.windows.net/DimTrack/data"
    # )


#DimDate

In [0]:
df_date = (
    spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "parquet")
        .option(
            "cloudFiles.schemaLocation",
            "abfss://silver@hariazureproject.dfs.core.windows.net/DimDate/schema"
        )
        .option(
            "checkpointLocation",
            "abfss://silver@hariazureproject.dfs.core.windows.net/DimDate/checkpoint"
        )
        .load("abfss://bronze@hariazureproject.dfs.core.windows.net/DimDate")
)


In [0]:
df_date = df_date.drop(df_date['_rescued_data'])


In [0]:
df_date.writeStream.format("delta") \
    .outputMode("append") \
    .option(
        "checkpointLocation",
        "abfss://silver@hariazureproject.dfs.core.windows.net/DimDate/checkpoint"
    ) \
    .trigger(once=True) \
    .option("path","abfss://silver@hariazureproject.dfs.core.windows.net/DimDate/data")\
    .toTable("spotify_cata.silver.DimDate")
    # .start(
    #     "abfss://silver@hariazureproject.dfs.core.windows.net/DimDate/data"
    # )


#FactStream

In [0]:
df_fact = (
    spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "parquet")
        .option(
            "cloudFiles.schemaLocation",
            "abfss://silver@hariazureproject.dfs.core.windows.net/FactStream/schema"
        )
        .option(
            "checkpointLocation",
            "abfss://silver@hariazureproject.dfs.core.windows.net/FactStream/checkpoint"
        )
        .load("abfss://bronze@hariazureproject.dfs.core.windows.net/FactStream")
)


In [0]:
df_fact = df_fact.drop(df_fact['_rescued_data'])


df_fact.writeStream.format("delta") \
    .outputMode("append") \
    .option(
        "checkpointLocation",
        "abfss://silver@hariazureproject.dfs.core.windows.net/FactStream/checkpoint"
    ) \
    .trigger(once=True) \
    .option("path","abfss://silver@hariazureproject.dfs.core.windows.net/FactStream/data")\
    .toTable("spotify_cata.silver.FactStream")
    # .start(
    #     "abfss://silver@hariazureproject.dfs.core.windows.net/FactStream/data"
    # )
